In [1]:
import sys
import os
import time
import datetime as dt
import math
import numpy as np 
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import butter,filtfilt
from scipy import interpolate
from scipy.interpolate import interp1d
import pingouin as pg

In [2]:
# OLD 
def nannotime(row,shift,dshift):
    # function to generate a row of nans with caveats for time fields, which are shifted by a designated ammount 
    r = row.copy()
    cols = r.index
    for c in cols:
        if not c.startswith('time'):
            if not c.startswith('datetime'):
                r[c] = np.nan
            if c.startswith('datetime'):
                r[c] = r[c] + dshift  
        if c.startswith('time'):
            r[c] = r[c] + shift     
    return r

def gap_nans(data,gap_t):
    # data is a pandas dataframe with columns called time and/or time_concert which is used to ID gaps greater than gap_T
    # gap_T is in milliseconds (ms)
    # rows of NaN data is added to non-time columns before the first sample, at edges of each gap, and after the last sample of data
    # these nans prevent interpolation errors through missing data
    cols = data.columns
    if 'time' in cols:
        time_col = 'time'
    if 'time_concert' in cols:
        time_col = 'time_concert'
    deltat = round(0.35*data[time_col].diff().median())
    dtdeltat =  pd.to_timedelta(deltat,unit = 'ms')
    
    data = data.append(nannotime(data.iloc[-1,:],deltat,dtdeltat),ignore_index=True)
    
    dt = data[time_col].diff()
    a = list(dt[dt>gap_t].index)
    a.sort(reverse=True)
    for gapi in a:
        data = data.append(nannotime(data.iloc[gapi-1,:],deltat,dtdeltat))
        data = data.append(nannotime(data.iloc[gapi,:],-deltat,dtdeltat))

    data = data.append(nannotime(data.iloc[0,:],-1,dtdeltat)).sort_values(time_col,ignore_index=True)
    return data

In [66]:
# NEW NO PD APPEND
def gap_nans(data,gap_t):
    # data is a pandas dataframe with columns called time and/or time_concert which is used to ID gaps greater than gap_T
    # gap_T is in milliseconds (ms)
    # rows of NaN data is added to non-time columns before the first sample, at edges of each gap, and after the last sample of data
    # these nans prevent interpolation errors through missing data
    cols = data.columns
    if 'time' in cols:
        time_col = 'time'
    if 'time_concert' in cols:
        time_col = 'time_concert'
        
    deltat = round(0.35*data[time_col].diff().median())
    dtdeltat =  pd.to_timedelta(deltat,unit = 'ms')
    dt = data[time_col].diff()    
    gapsi = np.array(dt[dt>gap_t].index)
    a = np.append(gapsi-1,dt.index[-1])
    gapsd = pd.DataFrame(index = a,columns = data.columns)
    for c in cols:
        if c.startswith('datetime'):
            gapsd[c] = data.loc[a,c]+dtdeltat
        if c.startswith('time'):
            gapsd[c] = data.loc[a,c]+deltat 

    a = np.append(gapsi,dt.index[0])
    gapsp = pd.DataFrame(index = a,columns = data.columns)
    for c in cols:
        if c.startswith('datetime'):
            gapsp[c] = data.loc[a,c]-dtdeltat
        if c.startswith('time'):
            gapsp[c] = data.loc[a,c]-deltat 

    data = pd.concat([gapsp,gapsd,data]).sort_values(time_col,ignore_index=True)
    return data

In [3]:
general_loc = 'C:/Users/finnu/OneDrive - Universitetet i Oslo/Documents/Mobile_data/'

public_mlmotion = general_loc + 'Public/Motion_ML/'
public_ax3 = general_loc +  'Public/Motion_AX3/'
public_Part = general_loc + 'Public/Participants/'
public_time = general_loc + 'Public/Timing/'

In [4]:
# set the sampling period for the concert in UTC 
Start_time = pd.to_datetime('2021-10-26 17:30:00+0000').replace(tzinfo=None)
Synch_time = pd.to_datetime('2021-10-26 17:48:06+0000').replace(tzinfo=None)
Mid_time = pd.to_datetime('2021-10-26 19:00:00+0000').replace(tzinfo=None)
End_time = pd.to_datetime('2021-10-26 20:30:00+0000').replace(tzinfo=None)

# concert times 10 Hz
ts_ct = np.arange(-1086,9715, 0.1)*1000 
dt_ct = Synch_time + pd.to_timedelta(ts_ct,unit='ms')


concert_times = pd.read_csv(public_time +'Events_Full.csv',sep=",") #SV alignment for beeps
concert_times['sequence'] = concert_times.index
concert_times.set_index('Datetime',inplace = True)
concert_times.iloc[:5]

,Time_s,Time_ms,Event,Event_Type,Time_Stream,sequence
Datetime,,,,,,
2021-10-26 17:36:35,-691.24,-691240,Clapping,Audience Action,946.00,0
2021-10-26 17:36:52,-674.12,-674120,Clapping Off,Audience Action,963.12,1
2021-10-26 17:36:53,-673.12,-673120,Danish,Speech,964.12,2
2021-10-26 17:37:10,-656.12,-656120,English,Speech,981.12,3
2021-10-26 17:37:20,-645.60,-645600,Danish,Speech,991.64,4


In [5]:
partDets = pd.read_csv(public_Part + 'Audience_Mapping_Public.csv')#,index_col = 0,header_col = 0)
# need only in person participants with aligned motion traces
partDets = partDets.loc[partDets['Attendance'] == 'A',:] # in person hall participant
alignedParts = partDets.loc[partDets['Motion_Quality'].isin(['Q','R']),:].reset_index(drop = True) # with aligned motion measurements
aligned_Part_Codes = list(alignedParts.Participant_Code.values)
alignedParts

,Participant_Code,Participant_Number,Attendance,Questionnaire_Language,Motion_Quality,Link_Conflicts,Seat_Zone,Location,Longitude_Truncated,Lattitude_Truncated,Offset_1,Offset_2,Watch_Alone
0,ADQ001,1,A,D,Q,0,6.0,NaN,NaN,NaN,0.14,0.16,0.0
1,ADQ002,2,A,D,Q,0,3.0,NaN,NaN,NaN,NaN,-0.36,0.0
2,ADQ003,3,A,D,Q,0,2.0,NaN,NaN,NaN,0.88,1.02,0.0
3,ADQ004,4,A,D,Q,0,4.0,NaN,NaN,NaN,0.44,0.62,1.0
4,ADQ005,5,A,D,Q,0,4.0,NaN,NaN,NaN,0.04,0.12,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,AEQ086,86,A,E,Q,0,5.0,NaN,NaN,NaN,NaN,0.34,0.0
81,AEQ087,87,A,E,Q,1,3.0,NaN,NaN,NaN,0.12,0.28,0.0
82,AEQ088,88,A,E,Q,0,2.0,NaN,NaN,NaN,0.10,0.06,0.0
83,AEQ089,89,A,E,Q,0,4.0,NaN,NaN,NaN,0.28,0.38,0.0


In [47]:
# load those motion traces 
# gather the aligned recordings for the live performance. Combining AX3 and ML data!
aud_df = pd.DataFrame(index = ts_ct)
kept_dev = []
gap_t = 1000
tic = time.time()
for i,participant in alignedParts.iloc[:10,:].iterrows():
    part = participant['Participant_Code']
    if part[2]=='Q': # Motionlab phone acc
        inst_d = pd.read_csv(public_mlmotion + part + '_Mobile_Motion.csv' )
    else: # AX3
        inst_d = pd.read_csv(public_ax3 + part + '_AX3_Motion.csv' )
    inst_d['datetime']=(pd.to_datetime(inst_d['datetime'])) # concert time stamps, confirm format
    datas = gap_nans(inst_d,1000)
    #get concert time
    resampled = pd.DataFrame(index = ts_ct)
    cols = datas.columns
    for col in cols[1:]:
        f = interpolate.interp1d(datas['time'], datas[col],fill_value='extrapolate')
        resampled[col] = f(ts_ct)
        
    # extract 1D accelerometer feature for easy comparison
    # and basic normalisation
    b = np.linalg.norm(resampled[['x','y','z']].diff().values,axis=1)
    if np.nanmedian(b)>0:
        aud_df[part] = b/np.nanmedian(b)
    else:
        aud_df[part] = b/np.nanmean(b)

print(time.time()-tic)
aud_df.index = dt_ct

C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[-1,:],deltat,dtdeltat),ignore_index=True)
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi-1,:],deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi,:],-deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pan

C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi-1,:],deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi,:],-deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi-1,:],deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat ins

C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi-1,:],deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi,:],-deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi-1,:],deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat ins

C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi,:],-deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi-1,:],deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi,:],-deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi,:],-deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi-1,:],deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi,:],-deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

...

C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi,:],-deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi-1,:],deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi,:],-deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi,:],-deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi-1,:],deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi,:],-deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi-1,:],deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi,:],-deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi-1,:],deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat ins

C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi-1,:],deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi,:],-deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(nannotime(data.iloc[gapi-1,:],deltat,dtdeltat))
C:\Users\finnu\AppData\Local\Temp\ipykernel_17936\966561879.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat ins

114.83612537384033


In [65]:
# load those motion traces 
# gather the aligned recordings for the live performance. Combining AX3 and ML data!
aud_df = pd.DataFrame(index = ts_ct)
kept_dev = []
gap_t = 1000
tic = time.time()
for i,participant in alignedParts.iloc[:10,:].iterrows():
    part = participant['Participant_Code']
    if part[2]=='Q': # Motionlab phone acc
        inst_d = pd.read_csv(public_mlmotion + part + '_Mobile_Motion.csv' )
    else: # AX3
        inst_d = pd.read_csv(public_ax3 + part + '_AX3_Motion.csv' )
    inst_d['datetime']=(pd.to_datetime(inst_d['datetime'])) # concert time stamps, confirm format
    datas = gap_nans2(inst_d,1000)
    #get concert time
    resampled = pd.DataFrame(index = ts_ct)
    cols = datas.columns
    for col in cols[1:]:
        f = interpolate.interp1d(datas['time'], datas[col],fill_value='extrapolate')
        resampled[col] = f(ts_ct)
        
    # extract 1D accelerometer feature for easy comparison
    # and basic normalisation
    b = np.linalg.norm(resampled[['x','y','z']].diff().values,axis=1)
    if np.nanmedian(b)>0:
        aud_df[part] = b/np.nanmedian(b)
    else:
        aud_df[part] = b/np.nanmean(b)
print(time.time()-tic)        
aud_df.index = dt_ct

27.077882766723633
